This is the Saildrone and CCMP collocation code. 


In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from netCDF4 import Dataset, date2index, num2date
import datetime as dt
from math import atan2
import xarray as xr

def get_ccmp(lyr,idyjl):
    iexist,ds=[]
    dir_ccmp='F:/data/sat_data/ccmp/v02.0/'
    d = dt.datetime(2010,1,1) + dt.timedelta(days=idyjl-1)
    day_of_year, imon, idym = d.timetuple().tm_yday, d.month, d.day
    syr, smon, sdym, sjdy=str(lyr),str(imon),str(idym),str(idyjl)
    fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
    ccmp_filename = dir_ccmp + 'Y' + syr + '/M' + smon.zfill(2) + fname_tem      
    exists = os.path.isfile(ccmp_filename)
    if exists==False:
        fname_tem='/CCMP_RT_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
        ccmp_filename = dir_ccmp + 'Y' + syr + '/M' + smon.zfill(2) + fname_tem      
        exists = os.path.isfile(ccmp_filename)
   # print(exists,ccmp_filename)
    if exists:       
        ds=xr.open_dataset(ccmp_filename,drop_variables=['nobs'])
        ds = ds.rename({'longitude':'lon','latitude':'lat'}) #, inplace = True)  
        ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon').sortby('lat')
        #ds['spd']=np.sqrt(ds.uwnd**2+ds.vwnd**2)
        iexist=1
    return ds, iexist

def get_ccmp_filename(date):
    dir_ccmp='F:/data/sat_data/ccmp/v02.0/'
#    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl-1)
    lyr, idyjl, imon, idym = date.dt.year.data, date.dt.dayofyear.data, date.dt.month.data, date.dt.day.data
    syr, smon, sdym, sjdy=str(lyr),str(imon),str(idym),str(idyjl)
    fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
    ccmp_filename = dir_ccmp + 'Y' + syr + '/M' + smon.zfill(2) + fname_tem      
    exists = os.path.isfile(ccmp_filename)
    if exists==False:
        fname_tem='/CCMP_RT_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
        ccmp_filename = dir_ccmp + 'Y' + syr + '/M' + smon.zfill(2) + fname_tem      
        exists = os.path.isfile(ccmp_filename)
   # print(exists,ccmp_filename)
    return ccmp_filename, exists


In [21]:
filename_collocation_data = 'F:/data/cruise_data/saildrone/baja-2018/ccmp_collocation_data.nc'
#filename_usv = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
filename_usv='f:/data/cruise_data/saildrone/baja-2018/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
#filename_usv='f:/data/cruise_data/saildrone/baja-2018/old_versions/saildrone-gen_5-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v2.1535585233403.nc'
ds_usv = xr.open_dataset(filename_usv)
ds_usv.close()
ds_usv = ds_usv.isel(trajectory=0)  
ds_usv = ds_usv.swap_dims({'obs':'time'})
ds_usv = ds_usv.rename({'longitude':'lon','latitude':'lat'})
#there are 6 nan values
ds_usv['lon'] = ds_usv.lon.interpolate_na(dim='time',method='linear')
ds_usv['lat'] = ds_usv.lat.interpolate_na(dim='time',method='linear')
ds_usv_subset = ds_usv.dropna(dim='time',subset={'UWND_MEAN'})   #get rid of all the nan
print(ds_usv_subset.UWND_MEAN[2000:2010].values)

[3.03 3.78 4.51 3.6  3.   3.91 4.96 4.16 4.41 4.65]


In [22]:
start_date = ds_usv.time.min()
end_date = ds_usv.time.max()
date = start_date.data
print(date)
date = date+np.timedelta64(3,'D')
print(date)


2018-04-11T18:00:00.000000000
2018-04-14T18:00:00.000000000


In [39]:
#create list of ccmp filenames to input
start_date,end_date = ds_usv.time.min(),ds_usv.time.max()
date,filelist = start_date,[]
while date<=(end_date+np.timedelta64(1,'D')):
    tem_filename, exists = get_ccmp_filename(date)
    if exists:
        filelist.append(tem_filename)
    date=date+np.timedelta64(1,'D')

In [37]:
#ds_sat,iexist = get_ccmp(2018,102)
#ccmp_path = 'F:/data/sat_data/ccmp/v02.0/Y2018/M04/'
ds_sat = xr.open_mfdataset(filelist,coords='minimal')
ds_sat = ds_sat.rename({'longitude':'lon','latitude':'lat'}) #, inplace = True)  
ds_sat = ds_sat.assign_coords(lon=(((ds_sat.lon + 180) % 360) - 180)).sortby('lon').sortby('lat')
ds_sat

<xarray.Dataset>
Dimensions:  (lat: 628, lon: 1440, time: 248)
Coordinates:
  * lon      (lon) float32 -179.875 -179.625 -179.375 ... 179.625 179.875
  * lat      (lat) float32 -78.375 -78.125 -77.875 ... 77.875 78.125 78.375
  * time     (time) datetime64[ns] 2018-04-11 ... 2018-06-11T18:00:00
Data variables:
    uwnd     (time, lat, lon) float32 dask.array<shape=(248, 628, 1440), chunksize=(4, 628, 1440)>
    vwnd     (time, lat, lon) float32 dask.array<shape=(248, 628, 1440), chunksize=(4, 628, 1440)>
    nobs     (time, lat, lon) float32 dask.array<shape=(248, 628, 1440), chunksize=(4, 628, 1440)>
Attributes:
    contact:                    Remote Sensing Systems, support@remss.com
    Conventions:                CF-1.6
    data_structure:             grid
    title:                      RSS CCMP_RT V2.1 derived surface winds (Level...
    history:                    20180426T175522ZZ - netCDF generated from ori...
    description:                RSS VAM 6-hour analyses starting fr

In [44]:
#make ds_sat smaller by slicing to only relevant lat/lon region
subset = ds_sat.sel(lon=slice(ds_usv.lon.min().data,ds_usv.lon.max().data),lat=slice(ds_usv.lat.min().data,ds_usv.lat.max().data))
#now we have a smaller dataset so make chuncks same size as array otherwise interp will throw an error
subset_chuncked = subset.chunk({'time': 248, 'lat': 39, 'lon':40})
subset_chuncked

-78.375 78.375 -179.875 179.875
28.0176832 37.8174912 -125.55297279999999 -115.5226624


<xarray.Dataset>
Dimensions:  (lat: 39, lon: 40, time: 248)
Coordinates:
  * lon      (lon) float32 -125.375 -125.125 -124.875 ... -115.875 -115.625
  * lat      (lat) float32 28.125 28.375 28.625 28.875 ... 37.125 37.375 37.625
  * time     (time) datetime64[ns] 2018-04-11 ... 2018-06-11T18:00:00
Data variables:
    uwnd     (time, lat, lon) float32 dask.array<shape=(248, 39, 40), chunksize=(248, 39, 40)>
    vwnd     (time, lat, lon) float32 dask.array<shape=(248, 39, 40), chunksize=(248, 39, 40)>
    nobs     (time, lat, lon) float32 dask.array<shape=(248, 39, 40), chunksize=(248, 39, 40)>
Attributes:
    contact:                    Remote Sensing Systems, support@remss.com
    Conventions:                CF-1.6
    data_structure:             grid
    title:                      RSS CCMP_RT V2.1 derived surface winds (Level...
    history:                    20180426T175522ZZ - netCDF generated from ori...
    description:                RSS VAM 6-hour analyses starting from the NC

In [51]:
ds_sat2 = subset_chuncked.interp(lat=ds_usv_subset.lat,lon=ds_usv_subset.lon,time=ds_usv_subset.time,method='linear')
#print(ds_sat2.uwnd.sel(time=slice('2018-04-12','2018-04-12')).values)
ds_sat2

<xarray.Dataset>
Dimensions:     (time: 17613)
Coordinates:
    lat         (time) float64 37.81 37.81 37.81 37.81 ... 37.78 37.78 37.78
    lon         (time) float64 -122.4 -122.4 -122.4 ... -122.3 -122.3 -122.3
  * time        (time) datetime64[ns] 2018-04-11T18:00:00 ... 2018-06-11T05:34:00
    trajectory  float32 1002.0
Data variables:
    uwnd        (time) float32 dask.array<shape=(17613,), chunksize=(17613,)>
    vwnd        (time) float32 dask.array<shape=(17613,), chunksize=(17613,)>
    nobs        (time) float32 dask.array<shape=(17613,), chunksize=(17613,)>
Attributes:
    contact:                    Remote Sensing Systems, support@remss.com
    Conventions:                CF-1.6
    data_structure:             grid
    title:                      RSS CCMP_RT V2.1 derived surface winds (Level...
    history:                    20180426T175522ZZ - netCDF generated from ori...
    description:                RSS VAM 6-hour analyses starting from the NCE...
    summary:      

In [53]:
ds_sat_xr = ds_sat2.to_array(name='uwnd')
ds_sat_xr

<xarray.DataArray 'uwnd' (variable: 3, time: 17613)>
dask.array<shape=(3, 17613), dtype=float32, chunksize=(1, 17613)>
Coordinates:
    lat         (time) float64 37.81 37.81 37.81 37.81 ... 37.78 37.78 37.78
    lon         (time) float64 -122.4 -122.4 -122.4 ... -122.3 -122.3 -122.3
  * time        (time) datetime64[ns] 2018-04-11T18:00:00 ... 2018-06-11T05:34:00
    trajectory  float32 1002.0
  * variable    (variable) <U4 'uwnd' 'vwnd' 'nobs'
Attributes:
    contact:                    Remote Sensing Systems, support@remss.com
    Conventions:                CF-1.6
    data_structure:             grid
    title:                      RSS CCMP_RT V2.1 derived surface winds (Level...
    history:                    20180426T175522ZZ - netCDF generated from ori...
    description:                RSS VAM 6-hour analyses starting from the NCE...
    summary:                    CCMP_RT V2.1 has been created using the same ...
    institute_id:               RSS
    institution:           

In [50]:
(ds_sat2.uwnd-ds_usv.UWND_MEAN).mean()

<xarray.DataArray ()>
dask.array<shape=(), dtype=float64, chunksize=()>
Coordinates:
    trajectory  float32 1002.0

In [ ]:
ds_sat2.uwnd[327]

In [ ]:
lats_usv=ds_usv.latitude.values
lons_usv=ds_usv.longitude.values
skin=ds_usv.TEMP_IR_UNCOR_MEAN.values
tair=ds_usv.TEMP_AIR_MEAN.values
bulk1=ds_usv.TEMP_CTD_MEAN.values
bulk2=ds_usv.TEMP_O2_MEAN.values
uspd=ds_usv.UWND_MEAN.values
vspd=ds_usv.VWND_MEAN.values
wing=ds_usv.HDG_WING.values
wing_ang=ds_usv.WING_ANGLE.values
pitch=ds_usv.PITCH.values
heading=ds_usv.HDG_WING.values
yaw_heading=ds_usv.HDG.values
roll=ds_usv.ROLL.values
tdim=len(skin)
wdir=np.zeros(tdim)
for i in range(0,tdim):
    wdir[i]=atan2(vspd[i],uspd[i])*180/3.14159
wspd=(uspd**2+vspd**2)**.5

In [ ]:
ds_sat = get_ccmp(2018,170)


In [ ]:
#get lat/lon from random file, it doesn't change
ds_sat = get_ccmp(2003,1)
lats_sat = ds_sat.lat
lons_sat = ds_sat.lon
ds_sat.close()
#initialize variables
latli_sv=-99
lonli_sv=-99
ihr_sv=-99
col_count=0  #initialize
gsst=np.zeros(tdim)
gsst_num=np.zeros(tdim)
#get goes data collocated with usv positions and times
for i in range(0,10): #tdim):
    latli = np.argmin( np.abs( lats_sat - lats_usv[i] ) )
    lonli = np.argmin( np.abs( lons_sat - lons_usv[i] ) )
    idy = ds_usv.time[i].dt.day.data        
    if (latli!=latli_sv) or (lonli_sv!=lonli) or (idy_sv!=idy):  #need to read in new data
        ds_sat = get_ccmp(ds_usv.time[i].dt.year.data,ds_usv.time[i].dt.dayofyear.data)
        col_count += 1
        gsst[i] = sat_sst
        gsst_num[i] = col_count  #this is to keep track of when a new file read in or new collocation point
        nc.close()
        latli_sv=latli
        lonli_sv=lonli
        ihr_sv=ds_usv.time[i].dt.hour.data
        print(i,tdim,lonli.data, latli.data,ihr_sv,gsst[i])
    else:  #collocation is to same gsst point
        gsst[i] = sat_sst 
        gsst_num[i] = col_count  #this is to keep track of when a new file read in or new collocation point


In [ ]:
#put into xarray dataset similiar in format to usv data
gsst2=np.zeros((1,tdim))
gsst_tem=np.zeros((1,tdim))
gsst2[0,:]=gsst
data_goes = xr.DataArray(gsst2, coords={'trajectory': ds_usv.trajectory,'obs': ds_usv.obs}, dims=('trajectory', 'obs'))
gsst_tem[0,:]=gsst_num
num_goes = xr.DataArray(gsst_tem, coords={'trajectory': ds_usv.trajectory,'obs': ds_usv.obs}, dims=('trajectory', 'obs'))
xr_gsst = xr.Dataset({'goes_sst': data_goes, 'collocation_index': num_goes}, 
                     coords={'trajectory':ds_usv.trajectory, 'time':ds_usv.time, 'latitude':ds_usv.latitude, 'longitude':ds_usv.longitude})   


In [ ]:
xr_gsst



In [ ]:
#np.save(filename_goes_sst, gsst)